In [12]:
#Installing Libraties
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
# Load dataset
df = pd.read_csv("dataset.csv")

In [14]:
#column names
print(df.columns.tolist())

['Unnamed: 0', 'Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date', 'Purchase Address', 'Month', 'Sales', 'City', 'Hour']


In [15]:
# Preprocessing
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

In [16]:
# Check for nulls
print("Missing values:\n", df.isnull().sum())

Missing values:
 unnamed:_0          0
order_id            0
product             0
quantity_ordered    0
price_each          0
order_date          0
purchase_address    0
month               0
sales               0
city                0
hour                0
dtype: int64


In [17]:
# Remove rows with missing values
df.dropna(inplace=True)

In [18]:
# Convert order_date to datetime
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

In [19]:
# Drop rows with invalid dates
df = df.dropna(subset=['order_date'])

In [20]:
# 🔹 Calculated Fields
df['quantity'] = pd.to_numeric(df['quantity_ordered'], errors='coerce')
df['unit_price'] = pd.to_numeric(df['price_each'], errors='coerce')

In [21]:
# Revenue (already exists in 'sales', but we recalculate for clarity)
df['revenue'] = df['quantity'] * df['unit_price']

In [22]:
# Estimate cost (assume 70% of unit price for cost if cost not available)
df['cost_price'] = df['unit_price'] * 0.7
df['cost'] = df['quantity'] * df['cost_price']
df['profit'] = df['revenue'] - df['cost']

# Aggregation

In [23]:
# Region = city, Product = product
region_product_sales = df.groupby(['city', 'product'])[['revenue', 'profit', 'quantity']].sum().reset_index()
region_product_sales.to_csv("region_product_sales.csv", index=False)

In [24]:
# Monthly revenue & profit trend
df['month'] = df['order_date'].dt.to_period('M')
monthly_sales = df.groupby('month')[['revenue', 'profit']].sum().reset_index()
monthly_sales['month'] = monthly_sales['month'].astype(str)
monthly_sales.to_csv("monthly_sales_trend.csv", index=False)

In [25]:
# 🔹 Monthly Profit Heatmap
df['year'] = df['order_date'].dt.year
df['month_name'] = df['order_date'].dt.strftime('%B')

monthly_heatmap = df.groupby(['year', 'month_name'])['profit'].sum().unstack(fill_value=0)

plt.figure(figsize=(12, 6))
sns.heatmap(monthly_heatmap, annot=True, fmt=".0f", cmap="YlGnBu")
plt.title("Monthly Profit Heatmap")
plt.tight_layout()
plt.savefig("monthly_profit_heatmap.png")
plt.close()

print("Saved: region_product_sales.csv, monthly_sales_trend.csv, and heatmap image.")

Saved: region_product_sales.csv, monthly_sales_trend.csv, and heatmap image.
